# Comonad and Comonad Transformers
Comonads are the categorical dual of monads. Whereas monads provide a computational context where we cannot 'see' into the container and only put things into it, a comonad explicity allows us to view it's contents but a way to put items into it is not provided.

In [73]:
-- side by side comparison of Monad definition and it's exact dual, Comonad
class (Functor m, Applicative m) => Monad m where
  return :: a -> m a
  (>>=) :: m a -> (a -> m b) -> m b
  bind :: (a -> m b) -> (m a -> m b) -- (=<<)
  join :: m (m a) -> m a
  
class Functor w => Comonad w where
  extract :: w a -> a
  extend :: (w a -> b) -> (w a -> w b) -- cobind
  duplicate :: w a -> w (w a)

## Examples of Comonads
Let's see some practical instances of Comonad to understand how they are useful.
### Store - the dual of State

In [77]:
data Store s a = Store (s -> a) s

instance Functor (Store s) where
  fmap f (Store g s) = Store (f . g) s
  
instance Comonad (Store s) where
  extract (Store accessor key) = accessor key
  extend f s = f <$> duplicate s
  duplicate (Store f s) = Store (Store f) s

Calling duplicate gives us a store of stores. What this means is that when we pass any key to the accessor function, we get back a store whose focus is that key.

Here are the utility functions for working with `Store`

In [78]:
-- | get the 'key' that this store is focused on
pos :: Store s a -> s
pos (Store f s) = s

-- | replace the focused key
seek :: s -> Store s a -> Store s a
seek s' (Store f s) = Store f s'

-- | modify the key
seeks :: (s -> s) -> Store s a -> Store s a
seeks f (Store g s) = Store g $ f s

-- | Peek at what the value would be for a different focus (key)
peek :: s -> Store s a -> a
peek s (Store f _) = f s

peeks :: (s -> s) -> Store s a -> a
peeks f (Store g s) = g $ f s

-- | Applies a functor-valued function to the stored value, and then uses the
--   new accessor to read the resulting focus.
experiment :: Functor f => (s -> f s) -> Store s a -> f a
experiment f (Store g s) = g <$> f s

Let's test it out with a simple store that is a mapping from an `Int` to another `Int`.

In [79]:
addOne :: Store Int Int
addOne = Store (+1) 5
extract addOne
peek 8 addOne
let f x = if x > 0 then Just (x^2) else Nothing
experiment f addOne
addOne' = seek (-2) addOne
experiment f addOne'

test = extend extract addOne -- id
extract test
test2 = extend (peek 2) addOne
extract test2
peek 8 test2

6

9

Just 26

Nothing

6

3

3

### Stream
A stream can be defined using `Store`

In [90]:
stream :: Store Int Int
stream = Store ([1,4,2,6,8] !!) 0
extract stream
extract $ seeks (+4) stream


1

8

## Comonad Transformers
Whereas Monad Transformers "wrap around" a core Monad (such as `IO`), Comonad Transformers embed a Comonad inside an outer Comonad.

In [102]:
class MonadTrans t where
  lift :: Monad m => m a -> t m a -- do something in the core monad then lift the result into the outer monad
  
class ComonadTrans t where
  lower :: Comonad w => t w a -> w a

-- Define a transformer for Store
data StoreT s w a = StoreT (w (s -> a)) s

instance Functor w => Functor (StoreT s w) where
  fmap f (StoreT wf s) = StoreT (fmap (f .) wf) s
  
instance Comonad w => Comonad (StoreT s w) where
  extract (StoreT wf s) = extract wf s
  
  -- extend :: Comonad w => (w a -> b) -> w a -> w b
  extend f (StoreT wf s) = StoreT (extend g wf) s where
      -- g :: Comonad w => w (s -> a) -> s -> b
      g wf' s' = f $ StoreT wf' s'

  duplicate (StoreT wf s) = StoreT (extend StoreT wf) s
  
instance ComonadTrans (StoreT s) where
  lower (StoreT wf s) = fmap ($ s) wf
  
inner = Store (\s -> (\s' -> s + s')) 3
test :: StoreT Int (Store Int) Int
test = StoreT inner 2
extract test
extract $ seek 5 $ lower test

5

7

## ListZipper and ListZipperT
Zippers in general are an example of comonads. We focus on a part while also having full access to the whole.

In [182]:
:ext MultiParamTypeClasses
:ext RecordWildCards
:ext InstanceSigs
:ext ScopedTypeVariables
:ext TypeSynonymInstances
:ext FlexibleInstances

class Indexable m i where
  (!) :: m a -> i -> a

data ListZipper a = ListZipper { list :: [a], index :: Int } deriving Show

shift :: Int -> ListZipper a -> ListZipper a
shift i ListZipper{..} = ListZipper list index' where
  index' = (index + i) `mod` (length list)

instance Indexable ListZipper Int where
  ListZipper{..} ! i = list !! i' where
    i' = (index + i) `mod` length list
    
instance Functor ListZipper where
  fmap f z@ListZipper{..} = z { list = fmap f list }
  
instance Comonad ListZipper where
  extract z = z ! (0 :: Int)
  -- run f on each cell, making it the focused cell
  extend f z = z { list = list' } where
    n = length . list $ z
    range = take n [0..]
    list' = map (f . flip shift z) range
  duplicate = extend id
  
-- Now a transformer version of the above

newtype ListZipperT w a = ListZipperT {
    runZipperT :: w (ListZipper a)
  }
  
shiftT :: Functor w => Int -> ListZipperT w a -> ListZipperT w a
shiftT i = ListZipperT . fmap (shift i) . runZipperT

instance Comonad w => Indexable (ListZipperT w) Int where
  z ! i = xs !! i' where
    ListZipper xs ind = extract . runZipperT $ z
    i' = (ind + i) `mod` length xs
  
instance Functor w => Functor (ListZipperT w) where
  fmap f = ListZipperT . (fmap . fmap) f . runZipperT
  
instance Comonad w => Comonad (ListZipperT w) where
  extract = extract . extract . runZipperT
  
  extend :: forall a b. (ListZipperT w a -> b) -> ListZipperT w a -> ListZipperT w b
  extend f = ListZipperT . extend go . runZipperT where
    f' :: w (ListZipper a) -> b
    f' = f . ListZipperT
    
    go :: w (ListZipper a) -> ListZipper b
    go wz = ListZipper ys i where
      ListZipper xs i = extract wz
      
      n = length xs
      range = take n [0..]
      
      shifted_wzs :: [w (ListZipper a)]
      shifted_wzs = map (\j -> fmap (shift j) wz) range -- shift each ListZipper that is contained in wz
      
      ys :: [b]
      ys = map f' shifted_wzs
      
  duplicate = extend id
  
instance ComonadTrans ListZipperT where
  -- produce a comonad of the focused elements in each ListZipper from a comonad of ListZippers
  lower = fmap extract . runZipperT
  
-- A 2D universe for cellular automata

type ZZ a = ListZipperT ListZipper a

instance Show a => Show (ZZ a) where
  show = show . runZipperT
  
instance Indexable (ListZipperT ListZipper) (Int, Int) where
  z ! (x, y) = extract . extract . shift y . runZipperT $ shiftT x z
  
fromList :: [[a]] -> ZZ a
fromList ys = ListZipperT (ListZipper xs 0) where
  xs = map (flip ListZipper 0) ys

toList :: ZZ a -> [[a]]
toList = list . fmap list . runZipperT

-- we can do horizontal and vertical indexing seperately without using the Indexable instance
index_horizontally :: Comonad w => Int -> ListZipperT w a -> a
index_horizontally i = (! i) . extract . runZipperT

index_vertically :: ComonadTrans t => Int -> t ListZipper a -> a
index_vertically i = (! i) . lower

## Conway's Game of Life

In [187]:
conway :: ZZ Char -> Char
conway zz = case count of
  2 -> extract zz
  3 -> '#'
  _ -> ' '
  where
  indices :: [(Int, Int)]
  indices = [(x, y) | x <- [-1..1], y <- [-1..1], (x, y) /= (0, 0)]
  
  vals = map (zz !) indices
  count = length $ filter (/= ' ') vals
  
life_step :: ZZ Char -> ZZ Char
life_step = extend conway

life_animation = iterate life_step